In [25]:
import pandas as pd
import numpy as np
import sympy as sp
from collections import namedtuple
from scipy.integrate import quad
from scipy.optimize import newton
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
import numpy_financial as npf
import matplotlib.pyplot as plt
from collections import namedtuple
from scipy.integrate import odeint

https://www.okonomiskeblikk.com/norges-banks-styringsrente-og-en-enkel-renteregel/

https://www.norges-bank.no/aktuelt/nyheter-og-hendelser/Signerte-publikasjoner/Staff-Memo/2022/sm-7-2022/



Dataframe

In [26]:
df = pd.read_excel('Siste.xlsx')
df_rente_kpi = pd.read_excel('siste_rente_kpi.xlsx')
bnp = pd.read_excel('BNP.xlsx')
kpi_jae = pd.read_excel('kpijae.xlsx')

In [27]:
n = 20

RENTE

In [28]:
pi_star = 0.02


In [29]:
def rente(r_star_list, pi_list, alpha_list, pi_star_list, y_list, y_star_list, inflasjonsmål):
    result = []

    for r_star, pi, alpha, pi_star, y, y_star in zip(r_star_list, pi_list, alpha_list, pi_star_list, y_list, y_star_list):
        i = r_star + pi + alpha * (pi - pi_star) + (1 - alpha) * (y - y_star) + inflasjonsmål
        i = np.clip(i, -0.5, 0.5)  # Clip the values to be within [-0.5, 0.5]
        result.append(i)

    return result

r_star

In [30]:
def generate_random_walk(n):
    random_walks = []
    # Generate a random starting value between -0.5 and 0.5
    start_value = np.random.uniform(-0.1, 0.1)

    # Generate a random walk with a step size of 0.1 from the last value
    steps = np.random.choice([-0.1, 0.1], size=12*n)
    random_walk = np.cumsum(np.concatenate([[start_value], steps]))

   # Ensure that the values stay within the range [-0.5, 0.5]
    random_walk = np.clip(random_walk, -0.5, 0.5)

    # Append the random walk to the list
    random_walks.append(random_walk)
    
    return random_walks

In [31]:
# Example usage:
r_star_value = generate_random_walk(n)         #  -0,5 - 0,5 i 
pi_value = 0.06             # inflasjon
alpha_value = 0.5           # tilfeldig alfavalue - ektraploere?
inflasjonsmål = 0.025       #Inflasjonsmålet - KPI-JAE
BNP_verdi = 100             #BNP data - faktisk verdi
BNP_trend = 90              #BNP data - trenden

#result = rente(r_star_value, pi_value, alpha_value, inflasjonsmål, BNP_verdi , BNP_trend, inflasjonsmål)
#print("Result:", result)

RENTE

In [39]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# Parameters
gamma = 0.02  # Rate of adjustment of policy interest rate
delta = 0.03  # Rate of adjustment of inflation
i_target = 0.02  # Inflation target
delay_i = 6  # Time delay for inflation adjustment
delay_r = 6  # Time delay for policy interest rate adjustment

# Differential equations with time delays
def model(y, t):
    i, r = y
    di_dt = delta * (r - i)
    # Use np.interp with mode='nearest' to handle edge cases
    dr_dt = gamma * (i_target - i) - gamma * (i - np.interp(t - delay_r, t, solution[:, 1]))
    return [di_dt, dr_dt]

# Initial conditions
i0 = 0.025  # Initial inflation
r0 = 0.0457  # Initial policy interest rate
y0 = [i0, r0]

# Time intervals
t = np.linspace(0, 25*12, 1000)

# Solve the ODE system
solution = odeint(model, y0, t)

# Create figure and axes
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot policy interest rate
color = 'tab:blue'
ax1.set_xlabel('Time')
ax1.set_ylabel('Policy Interest Rate', color=color)
ax1.plot(t, solution[:, 1], color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create another y-axis for inflation
ax2 = ax1.twinx()
color = 'tab:orange'
ax2.set_ylabel('Inflation', color=color)
ax2.plot(t, solution[:, 0], color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Title and grid
plt.title('Dynamics between policy interest rate and inflation over time with delays')
fig.tight_layout()
plt.grid(True)
plt.show()

ValueError: object of too small depth for desired array